In [7]:
def get_domain_name(hex,st):
    domain = ""
    len = int(hex[st:st+2],16)
    if len>191 :
        off = int(hex[st:st+4],16)
        off-=49152
        off *= 2
        dom,_ = get_domain_name(hex,off)
        return dom,st+4
    if len>63:
        print("Invalid data")
        return
    len*=2
    start = st+2
    while len :
        name = bytes.fromhex(hex[start:start+len]).decode("ASCII")
        domain+=name
        domain+="."
        start = start+len
        len = int(hex[start:start+2],16)
        len*=2
        start+=2
    return domain[:-1],start

def get_ip(hex):
    ip=""
    s=0
    for _ in range(4) :
        i = int(hex[s:s+2],16)
        s+=2
        ip+=str(i)+"."
    return ip[:-1]
        
        

def dns_parser(hex):
    domain = ""
    ip = ""
    if len(hex)<24:
        print("Invalid data")
        return
    q_r = int(hex[4],16)
    q_count = int(hex[8:12],16)
    a_count = int(hex[12:16],16)
    if q_count != 1 or a_count>1:
        print("Invalid data")
        return
    if q_r<7 and a_count>0:
        print("Invalid data")
        return
    if q_r > 7 :
        domain,end = get_domain_name(hex,24)
        end+=8
        dom,end = get_domain_name(hex,end)
        if dom != domain :
            print("Invalid data")
            return
        end+=20
        ip = get_ip(hex[end:])
        print(f"Domain name : {domain}")
        print(f"IP address : {ip}")
    else :
        domain,end = get_domain_name(hex,24)
        print(f"Domain name : {domain}")
        

# dns_packet_hex = "4c76010000010000000000010c74696d65736f66696e6469610a696e64696174696d657303636f6d00000100010000290200000000000000"
dns_packet_hex = "629f8180000100010000000103637365046969746d02616302696e0000010001c00c0001000100001f9600040a0608020000290200000000000000"
# dns_packet_hex = "00c88b57ec40ec2e98e9046b08004500004b527e40004011802f0a2a52f10a1800c2e36400350037875137920100000100000000000106636c69656e740764726f70626f7803636f6d00000100010000290200000000000000"
# dns_packet_hex = "ec2e98e9046b34e894fa3f5e08004500006b030400003e11f7c4c0a80001c0a80068003581c300577fa37baf818000010001000000010f7a2d7034322d696e7374616772616d046331307209696e7374616772616d03636f6d0000010001c00c000100010000000c00049df017ae0000290200000000000000"
# dns_packet_hex = "a0208180000100010000000105666f6e74730a676f6f676c656170697303636f6d0000010001c00c00010001000000ae00048efab64a0000290200000000000000"
dns_parser(dns_packet_hex)


Domain name : cse.iitm.ac.in
IP address : 10.6.8.2
